In [ ]:
%matplotlib widget

In [ ]:
import sys

import numpy as np

sys.path.insert(0, "..")
from seismicpro import Survey, SeismicDataset, VelocityCube
from seismicpro.src.metrics import PipelineMetric, pass_coords, pass_batch, pass_calc_args
from seismicpro.batchflow import Pipeline, V

## Вводная что вообще происходит

Я открыл пулл реквест с интерактивными картами, хочу чтобы вы потестировали новые фичи.

Порядок действий:
1. Прочитать описание пулл реквеста на гитхабе чтобы понять, что вообще поменялось
2. Пройтись по этому ноутбуку и потыкать в карты
3. Понять, удобен ли интерфейс и, если нет, предложить как его поменять
4. Начать смотреть в код. Оптимальная последовательность, на мой взгляд, такая:
    1. Батч + директория metrics/
    2. stacking_velocity
    3. interactive_plot_utils
    4. Все остальное
5. Более вдумчиво пройтись по этому ноутбуку, дергая разные параметры

Документации к коду пока нет, а если и есть - не читайте.

## Начнем с карт метрик по скоростному кубу

Подсуньте сюда куб из чатика:

In [ ]:
vc = VelocityCube("./broken_cube.txt")

In [ ]:
%%time
is_dec, max_acc, max_std, relative_var = vc.qc(10, times=np.arange(0, 1500, 2), bar=True)

Скоростной куб умеет в QC - в результате по дефолту мы получим 4 карты метрик. Поcмотрим, что это за зверь

In [ ]:
type(is_dec)

Основной атрибут - metric_data - содержит все значения метрики и соответствующие ей координаты. Хранение такого атрибута позволяет переагрегировать карту налету

In [ ]:
is_dec.metric_data

Карта умеет себя рисовать:

In [ ]:
is_dec.plot(figsize=(5, 5))

Более того, она умеет делать это интерактивно:

In [ ]:
is_dec.plot(interactive=True)

То, что будет рисоваться справа по умолчанию определяет класс метрики, которая была посчитана. Хранится она тут:

In [ ]:
is_dec.metric

Важно различать класс метрики и инстанс метрики:
* класс описывает метрику как функцию: он хранит статикметод calc и некоторую мету в своих классатрибутах (например, min_value и max_value)
* инстанс метрики помнит контекст, в котором метрика считаталсь. Например, инстанс выше помнит, в каком окне и по каким скоростям считался QC куба. Именно это позволяет ей нарисовать что-то осмысленное справа по клику

Карта умеет себя агрегировать и разагрегировать обратно в любой момент по вызову метода `aggregate`:

In [ ]:
agg_is_dec = is_dec.aggregate(bin_size=10)
agg_is_dec.plot(interactive=True)

Если карта бинаризована - мы можем итерироваться по содержимому бина в правом графике.

Разагрегация карты в исходное состояние:

In [ ]:
agg_is_dec.aggregate().plot(interactive=True)

Любой процесс агрегации нынче работает на несколько порядков быстрее, чем раньше, что позволяет делать его по запросу, а не единожны при создании карты.

От карт можно отнаследоваться. Чаще всего это нужно для того, чтобы доопределить рисовалку и дополнительно обработать какие-либо события. Например, оконные метрики QC куба скоростей позволяют подсветить область, по которой они считались:

In [ ]:
relative_var.plot(interactive=True, plot_window=True)

In [ ]:
max_std.plot(interactive=True, plot_window=False)

## Карты и пайплайны

Подсуньте сюда любой сегвай:

In [ ]:
%%time
header_index = "FieldRecord"
header_cols = ["INLINE_3D", "CROSSLINE_3D", "offset", "SourceX", "SourceY", "GroupX", "GroupY", "CDP_X", "CDP_Y"]

sur = Survey("./CDP_4_VA.sgy", header_index=header_index, header_cols=header_cols, name="raw")
ds = SeismicDataset(surveys=sur)

Самый частый кейс - рассчитывать метрику побатчево, а затем сагрегировать все в карту. Старый путь через рассчет метрик как-то руками и последующий вызов `gather_metrics` остался, но он не позволит удобным образом организовать интерактивность карты. Новый путь выглядит следующим образом:
1. Отнаследоваться от `PipelineMetric` и переопределить:
    * `calc` - метод рассчет самой метрики
    * Любое количество методов отрисовки по клику, принимающих на вход `ax` + какую-то инфу о клике (см. ниже)
    * Добавить названия всех плоттеорв с предыдущего шага в классатрибьют-туплю `views` 
2. В пайплайне вызвать новый метод `calculate_metric`, передав в него саму метрику и все аргументы для ее расчета
3. Наслаждаться

Пример рассчета стандартного отклонения по газерам, умноженного на константу:

In [ ]:
class MyMetric(PipelineMetric):
    name = "std"
    min_value = 0
    max_value = None
    is_lower_better = None
    views = "plot"

    @staticmethod
    def calc(gather, mul):
        return mul * gather.data.std()

    @pass_calc_args
    def plot(gather, mul, ax, **kwargs):
        gather.plot(ax=ax, **kwargs)

In [ ]:
template_ppl = (Pipeline()
    .load(src="raw")
    .sort(src="raw", dst="sorted", by="offset")
    .calculate_metric(MyMetric, gather="sorted", mul=100, save_to=V("accumulator", mode="a"))
)
ppl = (ds >> template_ppl)

In [ ]:
%%time
ppl.next_batch(500)

In [ ]:
mmap = ppl.v("accumulator").construct_map()

In [ ]:
mmap.plot(interactive=True)

В чем профит? Алгоритм расчета метрики, отрисовки по клику и всякая мета хранятся в одном месте. Для того чтобы получить кликалку от вас не требуется почти ничего, все заведется из коробки.

В классе выше появился декоратор `pass_calc_args` - он указывает на то, что методу отрисовки помимо `ax` прилетят еще и все арги, использованные для расчета метрики. Есть и другие опции со вполне говорящими названиями:

In [ ]:
class MyMultiViewMetric(PipelineMetric):
    name = "std"
    min_value = 0
    max_value = None
    is_lower_better = None
    views = ("plot", "plot_sorted", "plot_batch", "print_coords")

    @staticmethod
    def calc(gather, mul):
        return mul * gather.data.std()

    @pass_calc_args
    def plot(gather, mul, ax, **kwargs):
        gather.plot(ax=ax, **kwargs)

    @pass_calc_args
    def plot_sorted(gather, mul, ax, **kwargs):
        gather.sort(by="offset").plot(ax=ax, **kwargs)

    @pass_batch
    def plot_batch(batch, ax, **kwargs):
        batch.raw[0].plot(ax=ax, title="from batch")

    @pass_coords
    def print_coords(coords, ax, **kwargs):
        ax.set_title(coords)

In [ ]:
template_ppl = (Pipeline()
    .load(src="raw")
    .sort(src="raw", dst="sorted", by="offset")
    .calculate_metric(MyMultiViewMetric, gather="sorted", mul=100, save_to=V("accumulator", mode="a"))
)
ppl = (ds >> template_ppl)
ppl.next_batch(500)
ppl.v("accumulator").construct_map().plot(interactive=True)

В правом верхнем углу правого плота появилась кнопка, переключающая вид.

Режим "Сашка заебал со своими наследованиями, хочу тупо лямбды пихать и чтоб и интерактивно, и заебись, и еще и минеты строчились впридачу" тоже реализован:

In [ ]:
template_ppl = (Pipeline()
    .load(src="raw")
    .sort(src="raw", dst="sorted", by="offset")
    .calculate_metric(lambda gather, mul: mul * gather.data.std(), "raw", 100, metric_name="std", save_to=V("accumulator", mode="a"))
)
ppl = (ds >> template_ppl)

In [ ]:
%%time
ppl.next_batch(500)

In [ ]:
mmap = ppl.v("accumulator").construct_map()

In [ ]:
mmap.plot(interactive=True, plot_component="raw")

В этом случае вам придется указать компоненту батча, которую хотите нарисовать. Батч будет взят из того же самого датасета и прогонится по пайплайну до момента вызова `calculate_metric`. И да, много компонент тоже можно:

In [ ]:
mmap.plot(interactive=True, plot_component=["raw", "sorted"])

## Что еще интерактивненького появилось?

### Отрисоква геометрии съемки: 

In [ ]:
sur.plot_geometry()

В левом верхнем углу - переключалка с шотов на ресиверы.

### Интерактивное спрямление сейсмограммы с заданной скоростью:

In [ ]:
sur.sample_gather().sort(by="offset").plot_nmo_correction()

В левом верхнем углу - переключалка на вид исходного газера.

### Постоение карты атрибута трасс:

In [ ]:
max_offset_map = sur.construct_attribute_map("offset", by="receiver", agg="max")
max_offset_map.plot(interactive=True, sort_by="offset")

Это скорее общий интерфейс получения карты, для Дани сделаем отдельные алиасы под то, что ему нужно.

### Интерактивные семблансы:

Подсуньте сюда любой сегвай по бинам:

In [ ]:
%%time
header_index = "INLINE_3D", "CROSSLINE_3D"
header_cols = ["INLINE_3D", "CROSSLINE_3D", "offset", "SourceX", "SourceY", "GroupX", "GroupY", "CDP_X", "CDP_Y"]
semb_sur = Survey("./CDP_4_VA.sgy", header_index=header_index, header_cols=header_cols, name="raw")

In [ ]:
%%time
gather = semb_sur.sample_gather().sort(by="offset")
semblance = gather.calculate_semblance(velocities=np.arange(1400, 5500, 100))
stacking_velocity = semblance.calculate_stacking_velocity()
residual_semblance = gather.calculate_residual_semblance(stacking_velocity)

In [ ]:
semblance.plot(stacking_velocity, interactive=True)

Толщина годографа соответствует размеру окна при подсчете сембланса. А кнопка в правом верхнем углу - переключение в спрямленный вид.

In [ ]:
residual_semblance.plot(interactive=True)